In [1]:
import openpyxl
import mne
import mnelab
import numpy
import matplotlib
import xgboost
import pyriemann
import scipy
import pickle
import numpy as np
import pandas as pd
import FeatureExtract # 自己提取时域频域特征的方法实现
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score
from matplotlib import pyplot

In [2]:
# 行列标识符,P300二分类转字符36分类用
chars = {'A': (1, 7), 'B': (1, 8), 'C': (1, 9), 'D': (1, 10), 'E': (1, 11), 'F': (1, 12), 
         'G': (2, 7), 'H': (2, 8), 'I': (2, 9), 'J': (2, 10), 'K': (2, 11), 'L': (2, 12), 
         'M': (3, 7), 'N': (3, 8), 'O': (3, 9), 'P': (3, 10), 'Q': (3, 11), 'R': (3, 12), 
         'S': (4, 7), 'T': (4, 8), 'U': (4, 9), 'V': (4, 10), 'W': (4, 11), 'X': (4, 12), 
         'Y': (5, 7), 'Z': (5, 8), '1': (5, 9), '2': (5, 10), '3': (5, 11), '4': (5, 12), 
         '5': (6, 7), '6': (6, 8), '7': (6, 9), '8': (6, 10), '9': (6, 11), '0': (6, 12), 
        }

In [5]:
# 根据普适最优通道组合做特征工程处理原始数据集，转换为检测P300的二分类数据集
# 执行五次转换五个被试者数据集
optimal_channels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13] #最优通道序号

# 加载原始训练集
raw_data = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S5/S5_train_data.xlsx')
raw_event = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S5/S5_train_event.xlsx')

# 加载测试集
test_data = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_data.xlsx')
test_event = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_event.xlsx')

# 脑电数据集转换为P300二分类数据集，含剔除非优通道、数据预处理、滤波、特征提取等
classify_dataset = []
test_dataset = []

# 带通滤波器上下界
freq_min, freq_max = 1, 30
# 调用8阶巴特沃斯带通滤波器
# b, a = scipy.signal.butter(8, [0.02,0.8], 'bandpass')

训练集
for sheet in raw_event.sheetnames:
    # 当前字符
    current_char = sheet[7]
    # print(current_char)
    table_event = raw_event[sheet]
    table_data = raw_data[sheet]
    # 剔除bad通道电压值
    for col_num in range(20):
        if col_num+1 in optimal_channels:
            pass
        else:
            table_data.delete_cols(col_num+1)
            # print('delete ',col_num+1,' colum')
    for row in range(raw_event[sheet].max_row):
        if table_event.cell(row+1, 1).value >= 1 and table_event.cell(row+1, 1).value <= 12:
            # data.shape: 275,-100ms~1000ms时间窗口20个电压的均值
            # feature: 26
            feature = []
            data = []
            for i in range(len(optimal_channels)):
                channel = []
                for j in range(275):
                    channel.append(table_data.cell(table_event.cell(row+1, 2).value-24+j, i+1).value)
                data.append(channel)
                # 带通滤波
                # filter_data = scipy.signal.filtfilt(b, a, data[][i])
                filter_data = data[i]
                # 16个时域特征
                feature.append(FeatureExtract.mean_fea(filter_data))
                feature.append(FeatureExtract.rms_fea(filter_data))
                feature.append(FeatureExtract.sr_fea(filter_data))
                feature.append(FeatureExtract.am_fea(filter_data))
                feature.append(FeatureExtract.skew_fea(filter_data))
                feature.append(FeatureExtract.kurt_fea(filter_data))
                feature.append(FeatureExtract.max_fea(filter_data))
                feature.append(FeatureExtract.min_fea(filter_data))
                feature.append(FeatureExtract.pp_fea(filter_data))
                feature.append(FeatureExtract.var_fea(filter_data))
                feature.append(FeatureExtract.waveform_index(filter_data))
                feature.append(FeatureExtract.peak_index(filter_data))
                feature.append(FeatureExtract.impluse_factor(filter_data))
                feature.append(FeatureExtract.tolerance_index(filter_data))
                feature.append(FeatureExtract.skew_index(filter_data))
                feature.append(FeatureExtract.kurt_index(filter_data))
                # 10个频域特征
                feature.append(FeatureExtract.fft_mean(filter_data))
                feature.append(FeatureExtract.fft_var(filter_data))
                feature.append(FeatureExtract.fft_std(filter_data))
                feature.append(FeatureExtract.fft_entropy(filter_data))
                feature.append(FeatureExtract.fft_energy(filter_data))
                feature.append(FeatureExtract.fft_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_mean(filter_data))
                feature.append(FeatureExtract.fft_shape_std(filter_data))
                feature.append(FeatureExtract.fft_shape_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_kurt(filter_data))
            # 如果行或者列闪烁则添加P300真值标签， 否则添加负标签
            if table_event.cell(row+1, 1).value == chars[current_char][0] or table_event.cell(row+1, 1).value == chars[current_char][1]:
                feature.append(table_event.cell(row+1, 1).value)
            else:
                feature.append(0)
            classify_dataset.append(feature)
print((np.array(classify_dataset)).shape)
df = pd.DataFrame(classify_dataset)
df.to_csv('../data/semi/s5_selected_train.csv')

# 测试集
char_test = ['M', 'F', '5', '2', 'I']
ITER = 0
for sheet in test_event.sheetnames:
    # 完成前五个字符的sheet则退出
    if ITER >=5:
        break
    # 当前字符
    current_char = char_test[ITER]
    ITER += 1
    # print(current_char)
    table_event = test_event[sheet]
    table_data = test_data[sheet]
    # 剔除bad通道电压值
    for col_num in range(20):
        if col_num+1 in optimal_channels:
            pass
        else:
            table_data.delete_cols(col_num+1)
            # print('delete ',col_num+1,' colum')
    for row in range(test_event[sheet].max_row):
        if table_event.cell(row+1, 1).value >= 1 and table_event.cell(row+1, 1).value <= 12:
            # data.shape: 275,-100ms~1000ms时间窗口20个电压的均值
            # feature: 26
            feature = []
            data = []
            for i in range(len(optimal_channels)):
                channel = []
                for j in range(275):
                    channel.append(table_data.cell(table_event.cell(row+1, 2).value-24+j, i+1).value)
                data.append(channel)
                # 带通滤波
                # filter_data = scipy.signal.filtfilt(b, a, data[][i])
                filter_data = data[i]
                # 16个时域特征
                feature.append(FeatureExtract.mean_fea(filter_data))
                feature.append(FeatureExtract.rms_fea(filter_data))
                feature.append(FeatureExtract.sr_fea(filter_data))
                feature.append(FeatureExtract.am_fea(filter_data))
                feature.append(FeatureExtract.skew_fea(filter_data))
                feature.append(FeatureExtract.kurt_fea(filter_data))
                feature.append(FeatureExtract.max_fea(filter_data))
                feature.append(FeatureExtract.min_fea(filter_data))
                feature.append(FeatureExtract.pp_fea(filter_data))
                feature.append(FeatureExtract.var_fea(filter_data))
                feature.append(FeatureExtract.waveform_index(filter_data))
                feature.append(FeatureExtract.peak_index(filter_data))
                feature.append(FeatureExtract.impluse_factor(filter_data))
                feature.append(FeatureExtract.tolerance_index(filter_data))
                feature.append(FeatureExtract.skew_index(filter_data))
                feature.append(FeatureExtract.kurt_index(filter_data))
                # 10个频域特征
                feature.append(FeatureExtract.fft_mean(filter_data))
                feature.append(FeatureExtract.fft_var(filter_data))
                feature.append(FeatureExtract.fft_std(filter_data))
                feature.append(FeatureExtract.fft_entropy(filter_data))
                feature.append(FeatureExtract.fft_energy(filter_data))
                feature.append(FeatureExtract.fft_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_mean(filter_data))
                feature.append(FeatureExtract.fft_shape_std(filter_data))
                feature.append(FeatureExtract.fft_shape_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_kurt(filter_data))
            # 如果行或者列闪烁则添加P300真值标签， 否则添加负标签
            if table_event.cell(row+1, 1).value == chars[current_char][0] or table_event.cell(row+1, 1).value == chars[current_char][1]:
                feature.append(table_event.cell(row+1, 1).value)
            else:
                feature.append(0)
            test_dataset.append(feature)
print((np.array(test_dataset)).shape)
df = pd.DataFrame(test_dataset)
df.to_csv('../data/semi/s1_selected_test.csv')

(300, 313)


In [4]:
# 加载上一步转换后数据集
dataset_s1 = pd.read_csv('../data/semi/s1_selected_train.csv')
dataset_s2 = pd.read_csv('../data/semi/s2_selected_train.csv')
dataset_s3 = pd.read_csv('../data/semi/s3_selected_train.csv')
dataset_s4 = pd.read_csv('../data/semi/s4_selected_train.csv')
dataset_s5 = pd.read_csv('../data/semi/s5_selected_train.csv')
dataset = np.concatenate((dataset_s1, dataset_s2, dataset_s3, dataset_s4, dataset_s5), axis=0)

# 加载转换后测试集
dataset_test_s1 = pd.read_csv('../data/semi/s1_selected_test.csv')
dataset_test_s2 = pd.read_csv('../data/semi/s2_selected_test.csv')
dataset_test_s3 = pd.read_csv('../data/semi/s3_selected_test.csv')
dataset_test_s4 = pd.read_csv('../data/semi/s4_selected_test.csv')
dataset_test_s5 = pd.read_csv('../data/semi/s5_selected_test.csv')

# 随机删除过多的负样本,使正负样本1:1
delete_num = 0
while delete_num<2400:
    index = np.random.choice(dataset.shape[0],1,replace=True)
    # 只删负样本
    if np.squeeze(dataset[index])[-1] == 0.0:
        dataset = np.delete(dataset, index, axis=0)
        delete_num += 1

print(dataset.shape[0])
data = dataset[:, 1:313]
label = dataset[:, 313]
for index in range(len(label)):
    if label[index] != 0:
        label[index] = 1

# print(data.shape, len(label))

# 为全监督划分数据集为训练集，验证集，比例 4:1
data_train, data_eval, label_train, label_eval = train_test_split(data, label, test_size=0.2, random_state=1)
print('dataset split: ', len(data_train), ' ',len(data_eval))

1200
dataset split:  960   240


In [7]:
# 自主训练半监督，全监督作baseline对照
import xgboost

model = xgboost.XGBClassifier(
                            learning_rate=0.02,
                            n_estimators=90,
                            max_depth=4,
                            min_child_weight=1,
                            gamma=0,
                            subsample=1,
                            colsample_bytree=1,
                            objective='binary:logistic',
                            nthread=4,
                            scale_pos_weight=1,
                            seed=27)

# 全监督
clf_full = model.fit(data_train, label_train)
pred_full = clf_full.predict(data_eval)
print(classification_report(label_eval, pred_full))
print(" Full Accuracy: ", accuracy_score(label_eval,pred_full))

# 自学习半监督
# 定义一个无标签样本占训练集比例,去掉其标签
ratio = 0.5
rng = np.random.RandomState(10)

semi_label = np.copy(label_train)
semi_label[rng.rand(len(label_train)) < ratio] = -1

unlabeledX = data_train[semi_label == -1, :]
YTrue = label_train[semi_label == -1]

idx = np.where(semi_label != -1)[0]
labeledX = data_train[idx, :]
labeledY = semi_label[idx]

# 全监督
clf = model.fit(data_train, label_train)
score = accuracy_score(label_train, clf.predict(data_train))
test_score = accuracy_score(label_eval, clf.predict(data_eval))   # 测试集准确率
auc_score = roc_auc_score(label_eval, clf.predict(data_eval))
f1 = f1_score(label_eval, clf.predict(data_eval))
print("Training data size=", len(label_train), "Labeld accuracy= %.4f" % score, " , Unlabeled ratio=", 0,
      "ACC: %.4f" % test_score, 'AUC: %.4f' % auc_score, 'F1-score: %.4f' % f1)

# 半监督
clf = model.fit(labeledX, labeledY)
unlabeledY = clf.predict(unlabeledX)
unlabeledProb = clf.predict_proba(unlabeledX).max(axis=1)  # 预测为0和1的置信度，取大的
ratioInitial = 1 - (len(labeledY) / len(label_train))
score = accuracy_score(labeledY, clf.predict(labeledX))
test_score = accuracy_score(label_eval, clf.predict(data_eval))
auc_score = roc_auc_score(label_eval, clf.predict(data_eval))
f1 = f1_score(label_eval, clf.predict(data_eval))
print("iteration=",0, "Training data size=", len(labeledY), "Labeld accuracy=:  %.4f" % score, " , Unlabeled ratio=:  %.4f" % ratioInitial, "ACC: %.4f" % test_score, 'AUC: %.4f' % auc_score, 'F1-score: %.4f' % f1)

max_iter = 960
# 分类置信度阈值
probThreshold = 0.8

unlabeledXOrg = np.copy(unlabeledX)
YTrueOrg = np.copy(YTrue)

rr = []
it = []

i = 0
repeat = 1

log_Labeld_accuracy1 = []
log_Labeld_accuracy2 = []
log_Labeld_accuracy3 = []

while (i < max_iter and score > 0.01 and repeat<=10):

    lastscore = score
    ratio = 1 - (len(labeledY) / len(label_train))

    rr.append(ratio)
    it.append(i)

    labelidx = np.where(unlabeledProb > probThreshold)[0]
    unlabelidx = np.where(unlabeledProb <= probThreshold)[0]

    labeledX = np.vstack((labeledX, unlabeledX[labelidx, :]))  # 按照行顺序把数组给堆叠起来
    labeledY = np.hstack((labeledY, unlabeledY[labelidx]))
    unlabeledX = unlabeledX[unlabelidx, :]
    YTrue = label_train[unlabelidx]

    clf = model.fit(labeledX, labeledY)
    score = accuracy_score(labeledY, clf.predict(labeledX))
    test_score = accuracy_score(label_eval, clf.predict(data_eval))
    auc_score = roc_auc_score(label_eval, clf.predict(data_eval))
    f1 = f1_score(label_eval, clf.predict(data_eval))
    Labeld accuracy.append(score)
    print("iteration=",i+1, "Training data size=", len(labeledY), "Labeld accuracy= %.4f" % score, " , Unlabeled ratio= %.4f" % ratio, "ACC: %.4f" % test_score, 'AUC: %.4f' % auc_score, 'F1-score: %.4f' % f1)

    unlabeledY = clf.predict(unlabeledX)
    unlabeledProb = clf.predict_proba(unlabeledX).max(axis=1)

    i += 1
    if lastscore == score:
        repeat += 1
    else:
        repeat = 1
        


# 保存模型
file_semi = open('model/semi.pickle', 'wb')
clf_semi_model = pickle.dump(clf, file_semi)
# 验证集评估
pred_semi = clf.predict(data_eval)
print(classification_report(label_eval, pred_semi))
print("Semi Accuracy: ", accuracy_score(label_eval,pred_semi))

# 测试集前五个字符数据上评估
pred_test = clf.predict(data_eval)
print(classification_report(label_eval, pred_test))
print("Semi Accuracy: ", accuracy_score(label_eval,pred_test))

              precision    recall  f1-score   support

         0.0       0.48      0.44      0.46       129
         1.0       0.40      0.44      0.42       111

    accuracy                           0.44       240
   macro avg       0.44      0.44      0.44       240
weighted avg       0.44      0.44      0.44       240

 Full Accuracy:  0.44166666666666665
Training data size= 960 Labeld accuracy= 0.8635  , Unlabeled ratio= 0 ACC: 0.4417 AUC: 0.4417 F1-score: 0.4224
[1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1

iteration= 8 Training data size= 485 Labeld accuracy= 0.9485  , Unlabeled ratio= 0.4948 ACC: 0.4583 AUC: 0.4660 F1-score: 0.4922
iteration= 9 Training data size= 485 Labeld accuracy= 0.9485  , Unlabeled ratio= 0.4948 ACC: 0.4583 AUC: 0.4660 F1-score: 0.4922
iteration= 10 Training data size= 485 Labeld accuracy= 0.9485  , Unlabeled ratio= 0.4948 ACC: 0.4583 AUC: 0.4660 F1-score: 0.4922
              precision    recall  f1-score   support

         0.0       0.49      0.36      0.42       129
         1.0       0.43      0.57      0.49       111

    accuracy                           0.46       240
   macro avg       0.46      0.47      0.46       240
weighted avg       0.47      0.46      0.45       240

Semi Accuracy:  0.4583333333333333


In [11]:
# 针对测试集中char13~17检验方法

# 测试数据加载
raw_test_data = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_data.xlsx')
raw_test_event = openpyxl.load_workbook(u'/home/chris/predict_code/lyft/bci/data/S1/S1_test_event.xlsx')

# 测试数据预处理，特征提取, 5*5*12*521
# (即前5个有标签字符，每个五轮，每轮12次闪烁，20个通道，每个通道26个特征，共520特征,外加一个字符矩阵行列标识符)
# 标签即M、F、5、2、I
test_label = [113, 106, 131, 128, 109]
test_data = []

# 带通滤波器上下界
freq_min, freq_max = 1, 30
# 调用8阶带通滤波器
# b, a = scipy.signal.butter(8, [0.02,0.8], 'bandpass')

for sheet in raw_test_data.sheetnames:
    # 取sheet数据
    table_event = raw_test_event[sheet]
    table_data = raw_test_data[sheet]
    
    # 转换后数据,5轮次P300脑电处理后数据
    iter = 0 # 轮次数
    count = 0 # 计数
    char_data = [] # 多轮数据, 5*12*521
    iter_data = [] # 一轮数据, 12*521
    for row in range(raw_test_event[sheet].max_row):
        if table_event.cell(row+1, 1).value >= 1 and table_event.cell(row+1, 1).value <= 12:
            # data.shape: 275*20,-100ms~1000ms时间窗口
            # feature: 520
            feature = []
            data = []
            
            if count % 12 == 0:
                iter += 1
                count = 0
                char_data.append(iter_data)
                iter_data.clear()
            
            for i in range(20):
                channel = []
                for j in range(275):
                    channel.append(table_data.cell(table_event.cell(row+1, 2).value-24+j, i+1).value)
                data.append(channel)
            for i in range(20):
                # 带通滤波
                # filter_data = scipy.signal.filtfilt(b, a, data[i])
                filter_data = data[i]
                # 16个时域特征
                feature.append(FeatureExtract.mean_fea(filter_data))
                feature.append(FeatureExtract.rms_fea(filter_data))
                feature.append(FeatureExtract.sr_fea(filter_data))
                feature.append(FeatureExtract.am_fea(filter_data))
                feature.append(FeatureExtract.skew_fea(filter_data))
                feature.append(FeatureExtract.kurt_fea(filter_data))
                feature.append(FeatureExtract.max_fea(filter_data))
                feature.append(FeatureExtract.min_fea(filter_data))
                feature.append(FeatureExtract.pp_fea(filter_data))
                feature.append(FeatureExtract.var_fea(filter_data))
                feature.append(FeatureExtract.waveform_index(filter_data))
                feature.append(FeatureExtract.peak_index(filter_data))
                feature.append(FeatureExtract.impluse_factor(filter_data))
                feature.append(FeatureExtract.tolerance_index(filter_data))
                feature.append(FeatureExtract.skew_index(filter_data))
                feature.append(FeatureExtract.kurt_index(filter_data))
                # 10个频域特征
                feature.append(FeatureExtract.fft_mean(filter_data))
                feature.append(FeatureExtract.fft_var(filter_data))
                feature.append(FeatureExtract.fft_std(filter_data))
                feature.append(FeatureExtract.fft_entropy(filter_data))
                feature.append(FeatureExtract.fft_energy(filter_data))
                feature.append(FeatureExtract.fft_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_mean(filter_data))
                feature.append(FeatureExtract.fft_shape_std(filter_data))
                feature.append(FeatureExtract.fft_shape_skew(filter_data))
                feature.append(FeatureExtract.fft_shape_kurt(filter_data))
                # 字符矩阵行列标识符
            feature.append(table_event.cell(row+1, 1).value)
            iter_data.append(feature)
            count += 1
    test_data.append(char_data)     
print((np.array(test_data)).shape)    

(10, 5, 12, 521)


In [7]:
# P300二分类器模型加载,输出后5个字符
answer = []
with open('model/semi.pickle', 'rb') as fr:
    clf_p300 = pickle.load(fr)
    # 轮询后5个字符数据
    for i in range(len(test_data)):
        # size:5, 12, 521
        current_char_data = np.array(test_data[i])
        char_prob = []
        for j in range(len(current_char_data)):
            # size: 12, 521
            curr_iter_data = current_char_data[j]
            # 12次闪烁后脑电含P300波的概率， 12×(prob, 对应row或colum号)
            p300_prob = []
            for k in range(len(curr_iter_data)):
                prob_pred = (np.squeeze(clf_p300.predict_proba(np.expand_dims(curr_iter_data[k][0:-2], 0))))[0]
                # 行列标识符和对应该行列闪烁后有无P300的概率组成元组
                p300_prob.append((prob_pred, curr_iter_data[k][-1]))
            char_prob.append(p300_prob)
        # print((np.array(char_prob)))
        # P300二分类检测完成，求字符
        prob = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] # 初始各行或列闪烁后有无P300概率
        for a in range(len(char_prob)):
            p300_iter = char_prob[a]
            # 多轮加权
            for b in range(len(p300_iter)):
                prob[int(p300_iter[b][1]-1)] += p300_iter[b][0]
            for index in range(len(prob)):
                prob[index] = prob[index]/5.0
        # print(prob)
        # 求出对应行列标识符
        char_row = np.argmax(prob[0:6])
        char_col = np.argmax(prob[6:])
        # 基于行列标识符，按值找键查字符矩阵标识符字典
        answer.append([k for k,v in chars.items() if v==(char_row+1, char_col+7)])
        print((char_row+1, char_col+7), answer[i])

TypeError: object of type 'Workbook' has no len()